In [1]:
from __future__ import print_function
import argparse
import os
import random

import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.layers.recurrent import LSTM
from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
dataset='cifar10'
dataroot='f1'
workers=1
batchSize=5
imageSize=32
nz=400
ngf=64
ndf=64
niter=25
lr=0.0002
beta1=0.5
cuda=True
ngpu=1
netG=''
netD=''
outf='f1'
manualSeed=10
try:
    os.makedirs(outf)
except OSError:
    pass


In [3]:
import os
I='hundred/'
ls1=os.listdir(I)
M='dct_3/'
ls2=os.listdir(M)

import scipy.misc as sm
import numpy as np
import scipy.io as si
names=[]
xx=400*3
lls2=len(ls2)
m=int(0.9*lls2)
X=np.zeros((m,xx))
Y=np.zeros((m,3*32*32))
Xtest=np.zeros((lls2-m,xx))
Ytest=np.zeros((lls2-m,3*32*32))
count=0
for ind,i in enumerate(ls2):
    if(ind<m):
        names.append(i)
        li=len(i)
        temp=i[0:li-4]
        #temp=i.split('__')[0]
        tmp1=si.loadmat(M+i)['xx'][0:20,0:20,:]
        X[ind,:]=np.ravel(tmp1)
        temp2=sm.imread(I+temp+'.png')/255.0
        Y[ind,:]=np.ravel(temp2)
    else:
        names.append(i)
        li=len(i)
        temp=i[0:li-4]
        #temp=i.split('__')[0]
        tmp1=si.loadmat(M+i)['xx'][0:20,0:20,:]
        Xtest[count,:]=np.ravel(tmp1)
        temp2=sm.imread(I+temp+'.png')/255.0
        Ytest[count,:]=np.ravel(temp2)
        count=count+1
    
    
    
    
    

In [4]:
import sklearn.preprocessing as sp
#X=X/np.max(X)
X=sp.normalize(X)    


In [5]:
inps=3*imageSize*imageSize

In [6]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(1000, input_dim=xx, activation='sigmoid'))
	model.add(Dense(1000,activation='relu'))
	model.add(Dense(1000,activation='relu'))
	model.add(Dense(inps, activation='sigmoid'))
	# Compile model
	return model

In [7]:
def discr():
	# create model
	model = Sequential()
	model.add(Dense(50, input_dim=inps, kernel_initializer='normal', activation='relu'))
	#model.add(Dense(100, kernel_initializer='normal',activation='relu'))
	model.add(Dense(2, kernel_initializer='normal',activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	return model

In [8]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
gen=larger_model()
dis=discr()
make_trainable(dis, False)

In [9]:
def mixed(x,y):
    model=Sequential()
    model.add(x)
    model.add(y)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
    

In [10]:
GAN=mixed(gen,dis)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 3072)              6278072   
_________________________________________________________________
sequential_2 (Sequential)    (None, 2)                 153752    
Total params: 6,431,824
Trainable params: 6,278,072
Non-trainable params: 153,752
_________________________________________________________________


In [11]:
bs=20
make_trainable(dis,True)

for epoch in range(100):
    for i in range(100):
        ind=np.random.permutation(X.shape[0])[0:bs]
        data=X[(ind)]        
        data2=Y[(ind)]
        gen_im=gen.predict(data)
        train_D = np.concatenate((data2, gen_im))
        y_D=np.zeros((2*bs,2))
        y_D[0:bs,1]=1
        y_D[bs:,0]=1
        d_loss  = dis.train_on_batch(train_D,y_D)
        #print(gen_im.shape)

        y_G=np.zeros((bs,2))
        y_G[:,1]=1
        g_loss=GAN.train_on_batch(data,y_G)
        if(i%40==0):
            print((epoch,i,d_loss,g_loss))
        #break
    #break

(0, 0, 0.70381862, 0.63413709)
(0, 40, 0.010591134, 4.5844765)
(0, 80, 0.0035380868, 5.9287281)
(1, 0, 2.393301, 0.12626612)
(1, 40, 0.018350352, 4.0110712)
(1, 80, 0.14128086, 2.5333147)
(2, 0, 1.9787139, 0.19761932)
(2, 40, 0.12707216, 2.8249216)
(2, 80, 0.040536169, 4.2396789)
(3, 0, 0.039859787, 4.5799894)
(3, 40, 0.022769835, 5.0002599)
(3, 80, 0.015993057, 5.0315223)
(4, 0, 0.013936128, 4.8767176)
(4, 40, 0.014440086, 4.564497)
(4, 80, 0.033783685, 3.8519776)
(5, 0, 0.08374279, 2.994828)
(5, 40, 0.081567578, 4.6259179)
(5, 80, 0.023534233, 4.8050342)
(6, 0, 0.019905902, 4.5895438)
(6, 40, 0.016638195, 4.8041296)
(6, 80, 0.018818231, 4.577322)
(7, 0, 0.04616759, 3.1283689)
(7, 40, 0.094320275, 4.4216008)
(7, 80, 0.0086503942, 5.4982972)
(8, 0, 0.0073509105, 5.5150895)
(8, 40, 0.007728016, 5.8909431)
(8, 80, 0.0042747804, 5.8849168)
(9, 0, 0.0061071813, 5.6922207)
(9, 40, 0.0054593892, 6.0872936)
(9, 80, 0.0033519105, 6.4394846)
(10, 0, 0.0027080197, 6.6832852)
(10, 40, 0.004587763

KeyboardInterrupt: 

In [12]:
import scipy.misc as sm
temp=gen.predict(Xtest[50:51,:])
sm.imsave('vap.png',temp.reshape((32,32,3)))
sm.imsave('vap2.png',Ytest[50:51].reshape((32,32,3)))
